In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import lagrange
datafile = "normalization_data.xls"
data = pd.read_excel(datafile,header=None)
max_min=(data-data.min())/(data.max()-data.min())#最小最大规范化
zero_mean=(data-data.mean())/data.std()#零-均值规范化
decimal=data/10**np.ceil(np.log10(data.abs().max()))#小数定标规范化
print(max_min)
print(zero_mean)
print(decimal)

          0         1         2         3
0  0.074380  0.937291  0.923520  1.000000
1  0.619835  0.000000  0.000000  0.850941
2  0.214876  0.119565  0.813322  0.000000
3  0.000000  1.000000  1.000000  0.563676
4  1.000000  0.942308  0.996711  0.804149
5  0.264463  0.838629  0.814967  0.909310
6  0.636364  0.846990  0.786184  0.929571
          0         1         2         3
0 -0.905383  0.635863  0.464531  0.798149
1  0.604678 -1.587675 -2.193167  0.369390
2 -0.516428 -1.304030  0.147406 -2.078279
3 -1.111301  0.784628  0.684625 -0.456906
4  1.657146  0.647765  0.675159  0.234796
5 -0.379150  0.401807  0.152139  0.537286
6  0.650438  0.421642  0.069308  0.595564
       0      1      2       3
0  0.078  0.521  0.602  0.2863
1  0.144 -0.600 -0.521  0.2245
2  0.095 -0.457  0.468 -0.1283
3  0.069  0.596  0.695  0.1054
4  0.190  0.527  0.691  0.2051
5  0.101  0.403  0.470  0.2487
6  0.146  0.413  0.435  0.2571


In [15]:
datafile2="discretization_data.xls"
data2 = pd.read_excel(datafile2)
data2 = data2['肝气郁结证型系数'].copy()
data2.values.reshape((len(data2),1))
k=4
d1 = pd.cut(data2,k,labels=range(k))#等宽离散化，各个类比依次命名为0,1,2,3
#等频率离散化
w=[1.0*i/k for i in range(k+1)]
w=data2.describe(percentiles=w)[4:4+k+1]
w[0]=w[0]*(1-1e-10)
d2=pd.cut(data2,w,labels=range(k))
from sklearn.cluster import KMeans
kmodel  =KMeans(n_clusters=k,n_jobs=4)#建立模型，n_jobs是并行数，一般等于CPU核心数
kmodel.fit(data2.values.reshape((len(data2),1)))#训练模型
c=pd.DataFrame(kmodel.cluster_centers_).sort_values(0)#输出聚类中心并且排序
w=c.iloc[1:].rolling(window=2,center=False).mean()
w=[0]+list(w[0]+[data2.max()])
d3 = pd.cut(data2,w,labels=range(k))

def cluster_plot(d,k):#自定义作图函数显示聚类结果
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8,3))
    for j in range(0,k):
        plt.plot(data2[d==j],[j for i in d[d==j]],'o')
    plt.ylim(-0.5,k-0.5)
    return plt
cluster_plot(d1,k).show()
cluster_plot(d2,k).show()
cluster_plot(d3,k).show()
data2.reshape

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\tile.py:130: RuntimeWarning: invalid value encountered in less
  if (np.diff(bins) < 0).any():


ValueError: Bin labels must be one fewer than the number of bin edges